In [13]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
import json
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_text_splitters import (
    Language, RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter )
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
def gather_markdown_documents(sources: str) -> str:
    loader = DirectoryLoader('.', glob=f'{sources}/*.md', loader_cls=TextLoader)
    docs = loader.load()

    return '\n\n'.join(doc.page_content for doc in docs)

In [3]:
print(gather_markdown_documents('source'))

웅진코웨이 정수기 사용 설명서 및 안전 수칙
------------------------

### 안전 주의사항

#### 위험 (생명에 위험이 있는 상황)

*   손상되었거나 헐거운 전원 콘센트를 사용하지 마세요. 감전 및 화재의 위험이 있습니다.
    
*   전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용하세요. 규격에 맞지 않는 전원에 연결하면 사고 위험이 있습니다.
    
*   전원 플러그를 잡아당겨 뽑거나 제품 이동 시 전원선을 끌지 마세요. 전원선 손상으로 감전이나 화재의 위험이 있습니다.
    
*   젖은 손으로 전원 플러그를 만지지 마세요. 감전의 위험이 있습니다.
    
*   전원 플러그를 무리하게 구부리거나 가공·훼손하지 마세요. 전원선이 손상되어 감전 또는 제품 고장의 원인이 될 수 있습니다.
    
*   전원 플러그를 연속으로 꽂았다가 빼지 마세요. 과열로 화재의 위험이 있습니다.
    
*   플러그의 핀과 접촉부에 먼지나 물 등 이물질이 묻어 있으면 깨끗이 닦아주세요. 이물질로 인해 감전의 위험이 있습니다.
    
*   하나의 멀티탭(문어발 콘센트)에 여러 전기제품을 동시에 연결하지 마세요. 과부하로 인한 화재 위험이 있습니다.
    
*   장기간 사용하지 않을 때는 전원 플러그를 뽑아두세요. 화재 발생이나 성능 저하를 예방할 수 있습니다.
    
*   전원 플러그나 전원선이 손상된 경우 임의로 수리하거나 그대로 사용하지 말고 서비스센터에 연락해 교체하세요. 그대로 사용하면 감전 및 화재 위험이 있습니다.
    
*   전열기구(히터 등) 가까이에 제품을 설치하지 마세요. 복사열로 인해 제품이 과열되어 화재가 발생할 수 있습니다.
    
*   원수 공급 연결은 반드시 찬물(냉수) 배관에만 하세요. 뜨거운 물(온수) 배관에 연결하면 제품이 파손되거나 사고의 원인이 됩니다.
    
*   제품을 임의로 분해, 개조하거나 내부를 열어서 수리하지 마세요. 전기 부품이 손상되어 감전 및

In [4]:
def split_markdown_by_headers(markdown_document: str, headers_to_split_on=None):
    if headers_to_split_on is None:
        headers_to_split_on = [('##', 'Header 2'), ('###', 'Header 3')]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, 
                                            strip_headers=False)
    
    return splitter.split_text(markdown_document)

In [5]:
markdown_document = gather_markdown_documents('source')
split_markdown_by_headers(markdown_document)

[Document(metadata={}, page_content='웅진코웨이 정수기 사용 설명서 및 안전 수칙\n------------------------'),
 Document(metadata={'Header 3': '안전 주의사항'}, page_content='### 안전 주의사항  \n#### 위험 (생명에 위험이 있는 상황)  \n*   손상되었거나 헐거운 전원 콘센트를 사용하지 마세요. 감전 및 화재의 위험이 있습니다.  \n*   전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용하세요. 규격에 맞지 않는 전원에 연결하면 사고 위험이 있습니다.  \n*   전원 플러그를 잡아당겨 뽑거나 제품 이동 시 전원선을 끌지 마세요. 전원선 손상으로 감전이나 화재의 위험이 있습니다.  \n*   젖은 손으로 전원 플러그를 만지지 마세요. 감전의 위험이 있습니다.  \n*   전원 플러그를 무리하게 구부리거나 가공·훼손하지 마세요. 전원선이 손상되어 감전 또는 제품 고장의 원인이 될 수 있습니다.  \n*   전원 플러그를 연속으로 꽂았다가 빼지 마세요. 과열로 화재의 위험이 있습니다.  \n*   플러그의 핀과 접촉부에 먼지나 물 등 이물질이 묻어 있으면 깨끗이 닦아주세요. 이물질로 인해 감전의 위험이 있습니다.  \n*   하나의 멀티탭(문어발 콘센트)에 여러 전기제품을 동시에 연결하지 마세요. 과부하로 인한 화재 위험이 있습니다.  \n*   장기간 사용하지 않을 때는 전원 플러그를 뽑아두세요. 화재 발생이나 성능 저하를 예방할 수 있습니다.  \n*   전원 플러그나 전원선이 손상된 경우 임의로 수리하거나 그대로 사용하지 말고 서비스센터에 연락해 교체하세요. 그대로 사용하면 감전 및 화재 위험이 있습니다.  \n*   전열기구(히터 등) 가까이에 제품을 설치하지 마세요. 복사열로 인해 제품이 과열되어 화재가 발생할 수 있습니다.  \n*   원수 공급 연결은 반드시 찬물(냉수) 배관에만 하세요. 뜨거운 물(온수) 배관에 연결하면 제품이 

In [6]:
def chunk_documents(documents, chunk_size: int = 500, chunk_overlap: int = 50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, 
                                                chunk_overlap=chunk_overlap)
    
    return splitter.split_documents(documents)

In [7]:
markdown_document = gather_markdown_documents('source')
md_header_splits = split_markdown_by_headers(markdown_document)
chunk_documents(md_header_splits)

[Document(metadata={}, page_content='웅진코웨이 정수기 사용 설명서 및 안전 수칙\n------------------------'),
 Document(metadata={'Header 3': '안전 주의사항'}, page_content='### 안전 주의사항  \n#### 위험 (생명에 위험이 있는 상황)  \n*   손상되었거나 헐거운 전원 콘센트를 사용하지 마세요. 감전 및 화재의 위험이 있습니다.  \n*   전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용하세요. 규격에 맞지 않는 전원에 연결하면 사고 위험이 있습니다.  \n*   전원 플러그를 잡아당겨 뽑거나 제품 이동 시 전원선을 끌지 마세요. 전원선 손상으로 감전이나 화재의 위험이 있습니다.  \n*   젖은 손으로 전원 플러그를 만지지 마세요. 감전의 위험이 있습니다.  \n*   전원 플러그를 무리하게 구부리거나 가공·훼손하지 마세요. 전원선이 손상되어 감전 또는 제품 고장의 원인이 될 수 있습니다.  \n*   전원 플러그를 연속으로 꽂았다가 빼지 마세요. 과열로 화재의 위험이 있습니다.  \n*   플러그의 핀과 접촉부에 먼지나 물 등 이물질이 묻어 있으면 깨끗이 닦아주세요. 이물질로 인해 감전의 위험이 있습니다.'),
 Document(metadata={'Header 3': '안전 주의사항'}, page_content='*   하나의 멀티탭(문어발 콘센트)에 여러 전기제품을 동시에 연결하지 마세요. 과부하로 인한 화재 위험이 있습니다.  \n*   장기간 사용하지 않을 때는 전원 플러그를 뽑아두세요. 화재 발생이나 성능 저하를 예방할 수 있습니다.  \n*   전원 플러그나 전원선이 손상된 경우 임의로 수리하거나 그대로 사용하지 말고 서비스센터에 연락해 교체하세요. 그대로 사용하면 감전 및 화재 위험이 있습니다.  \n*   전열기구(히터 등) 가까이에 제품을 설치하지 마세요. 복사열로 인해 제품이 과열되어 화재가 발생할 수 있습니다.  \n

In [8]:
def load_and_split(sources: str):
    markdown_document = gather_markdown_documents(sources)
    md_header_splits = split_markdown_by_headers(markdown_document)
    md_chunks = chunk_documents(md_header_splits)
    
    return md_header_splits, md_chunks

In [9]:
source_path = "./source"
md, elements = load_and_split(source_path)

In [10]:
md[0]

Document(metadata={}, page_content='웅진코웨이 정수기 사용 설명서 및 안전 수칙\n------------------------')

In [11]:
len(elements)

69

In [14]:
def load_and_split_pdf(pdf_dir):
    all_chunks = []
    all_docs = []

    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):
            filepath = os.path.join(pdf_dir, filename)
            
            loader = PyMuPDFLoader(filepath)
            docs = loader.load()
            all_docs.extend(docs)

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=200,
                chunk_overlap=50 )
            chunks = text_splitter.split_documents(docs)
            all_chunks.extend(chunks)

    return all_docs, all_chunks

In [15]:
source_path = "./source"
pdf, elements = load_and_split_pdf(source_path)
pdf

[Document(metadata={'producer': 'WeasyPrint 65.1', 'creator': '', 'creationdate': '', 'source': './source/웅진코웨이 정수기 사용 설명서 및 안전 수칙.pdf', 'file_path': './source/웅진코웨이 정수기 사용 설명서 및 안전 수칙.pdf', 'total_pages': 11, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='웅진코웨이 정수기 사용 설명서 및 안전 수칙\n안전 주의사항\n위험 (생명에 위험이 있는 상황)\n손상되었거나 헐거운 전원 콘센트를 사용하지 마세요. 감전 및 화재의 위험이 있습니다.\n전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용하세요. 규격에 맞지 않는 전원에 연결하면 사고\n위험이 있습니다.\n전원 플러그를 잡아당겨 뽑거나 제품 이동 시 전원선을 끌지 마세요. 전원선 손상으로 감전이나 화재의 위험이\n있습니다.\n젖은 손으로 전원 플러그를 만지지 마세요. 감전의 위험이 있습니다.\n전원 플러그를 무리하게 구부리거나 가공·훼손하지 마세요. 전원선이 손상되어 감전 또는 제품 고장의 원인이\n될 수 있습니다.\n전원 플러그를 연속으로 꽂았다가 빼지 마세요. 과열로 화재의 위험이 있습니다.\n플러그의 핀과 접촉부에 먼지나 물 등 이물질이 묻어 있으면 깨끗이 닦아주세요. 이물질로 인해 감전의 위험이\n있습니다.\n하나의 멀티탭(문어발 콘센트)에 여러 전기제품을 동시에 연결하지 마세요. 과부하로 인한 화재 위험이 있습니\n다.\n장기간 사용하지 않을 때는 전원 플러그를 뽑아두세요. 화재 발생이나 성능 저하를 예방할 수 있습니다.\n전원 플러그나 전원선이 손상된 경우 임의로 수리

In [16]:
len(elements)

154

In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """The following is documentation related to the company Woongjin Coway.

You are a customer service representative at Woongjin Coway.  
Your task is to generate exactly **{num_questions}** customer questions and answers based solely on the provided document.

---------------------
{context}
---------------------

Please follow these strict instructions:

1. Generate exactly {num_questions} questions. No more, no less.
2. All questions must relate only to the information about Woongjin Coway provided above.
3. Vary the question types (e.g., safety, usage, maintenance, service policy, etc.).
4. All questions and answers must be written in **Korean** and should be complete sentences.
5. Each question should be paired with a corresponding answer that is accurate and grounded strictly in the document.
6. The output should be in **JSON format**, using separate JSON objects for each pair.
7. Do not use a list structure or array in the JSON. Just output individual JSON objects separated by commas.

# Example output format:
```json
{{
  "QUESTION": "전원 플러그가 손상됐을 때는 어떻게 해야 하나요?",
  "ANSWER": "전원 플러그가 손상된 경우, 임의로 수리하지 말고 반드시 서비스센터에 연락하여 교체를 요청하세요."
}},
{{
  "QUESTION": "여러 제품을 멀티탭에 함께 연결해도 되나요?",
  "ANSWER": "한 개의 멀티 콘센트에 여러 제품을 동시에 사용하는 것은 과열이나 화재의 원인이 될 수 있으므로 피해야 합니다."
}},
{{
  "QUESTION": "제품 점검 시 전원은 어떻게 해야 하나요?",
  "ANSWER": "제품을 청소하거나 점검할 때는 전원 플러그를 반드시 뽑고 원수 밸브를 잠가야 합니다."
}}
"""
)

In [18]:
import json
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

def custom_json_parser(response):
    json_string = response.content.strip().removeprefix("```json\n").removesuffix("\n```").strip()
    json_string = f'[{json_string}]'
    return json.loads(json_string)

In [19]:
chain = (
    prompt
    | ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | custom_json_parser
)  # 체인을 구성합니다.

In [20]:
qa_pair = []

for element in elements:
    if element.page_content:
        try:
            result = chain.invoke(
                {"context": element.page_content, "domain": "service", "num_questions": "2"}
            )
            qa_pair.extend(result)
        except json.JSONDecodeError as e:
            print(f"[JSONDecodeError] 무시됨: {e}")
        except Exception as e:
            print(f"[기타 오류] {e}")

```json
{
  "QUESTION": "웅진코웨이 정수기를 사용할 때 전원 플러그는 어떤 콘센트에 꽂아야 하나요?",
  "ANSWER": "전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다."
},
{
  "QUESTION": "손상되었거나 헐거운 전원 콘센트를 사용하면 어떤 위험이 있나요?",
  "ANSWER": "손상되었거나 헐거운 전원 콘센트를 사용하면 감전 및 화재의 위험이 있습니다."
}
``````json
{
  "QUESTION": "전원선을 끌면서 제품을 이동하면 어떤 위험이 있나요?",
  "ANSWER": "전원선을 끌면서 제품을 이동하면 전원선이 손상되어 감전이나 화재의 위험이 있습니다."
},
{
  "QUESTION": "젖은 손으로 전원 플러그를 만지면 어떤 위험이 있나요?",
  "ANSWER": "젖은 손으로 전원 플러그를 만지면 감전의 위험이 있습니다."
}
``````json
{
  "QUESTION": "전원 플러그를 연속으로 꽂았다가 빼면 어떤 위험이 있나요?",
  "ANSWER": "전원 플러그를 연속으로 꽂았다가 빼면 과열로 인해 화재의 위험이 있습니다."
},
{
  "QUESTION": "플러그의 핀과 접촉부에 먼지가 묻어 있을 때 어떻게 해야 하나요?",
  "ANSWER": "플러그의 핀과 접촉부에 먼지나 물 등 이물질이 묻어 있으면 깨끗이 닦아주어야 합니다. 이물질로 인해 감전의 위험이 있기 때문입니다."
}
``````json
{
  "QUESTION": "장기간 사용하지 않을 때는 어떻게 해야 하나요?",
  "ANSWER": "장기간 사용하지 않을 때는 전원 플러그를 뽑아두어 화재 발생이나 성능 저하를 예방할 수 있습니다."
},
{
  "QUESTION": "전원선이 손상된 경우 어떻게 해야 하나요?",
  "ANSWER": "전원선이 손상된 경우 임의로 수리하거나 그대로 사용하지 말고 서비스센터에 연락해 교체해야 합니다."
}
``````json
{
  "QUEST

In [21]:
qa_pair[:3]

[{'QUESTION': '웅진코웨이 정수기를 사용할 때 전원 플러그는 어떤 콘센트에 꽂아야 하나요?',
  'ANSWER': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'},
 {'QUESTION': '손상되었거나 헐거운 전원 콘센트를 사용하면 어떤 위험이 있나요?',
  'ANSWER': '손상되었거나 헐거운 전원 콘센트를 사용하면 감전 및 화재의 위험이 있습니다.'},
 {'QUESTION': '전원선을 끌면서 제품을 이동하면 어떤 위험이 있나요?',
  'ANSWER': '전원선을 끌면서 제품을 이동하면 전원선이 손상되어 감전이나 화재의 위험이 있습니다.'}]

In [22]:
len(qa_pair)

308

In [23]:
import json

with open("./source/woongjin.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_pair:
        qa_modified = {
            "instruction": qa["QUESTION"],
            "input": "",
            "output": qa["ANSWER"],
        }
        f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")

In [24]:
prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Based on the query below, please create a **{num_questions}** similar sentence with the same meaning as the sentence
You are a CTO and customer center employee who knows all the information about Woongjin Coway.
Your mission is to augment a given dataset so that it has the same meaning but the same answer
You should not provide more or less questions than this number.
The purpose of this question is to evaluate the understanding of Woongjin Coway by customers or employees.
Answers to questions generated should be provided identically, and answers should use the data provided.
For example, input_text = "Question": 'Can I touch the power plug with wet hands?' and 'answer': If you have been asked, 'Do not touch the power plug with wet hands,' make a sentence that is similar to but has the same meaning as 'Can I touch the power plug with wet hands?' The answer is 'Do not touch the power plug with wet hands.'

Questions should be limited to similar but synonymous questions for the queries provided.
All questions and answers should be written in Korean, and answers should be provided in JSON format with questions and answers.
Do not use a list form in JSON format.
Your answer should be in complete sentences.

#Format:
json
    {{
        "messages": [
            {{ "role": "user", "content": "젖은 손으로 전원 플러그를 만져도 되나요?" }},
            {{ "role": "assistant", "content": "젖은 손으로 전원 플러그를 만지지 마세요." }}
        ]
    }},
    {{
        "messages": [
            {{ "role": "user", "content": "습기가 남은 손으로 전원 소켓에 손을 대도 괜찮나요?" }},
            {{ "role": "assistant", "content": "젖은 손으로 전원 플러그를 만지지 마세요." }}
        ]
    }}

"""
)

In [25]:
def custom_json_parser(response):
    raw = response.content.strip()
    raw = raw.removeprefix("```json\n").removesuffix("\n```").strip()
    array = f'[{raw}]'
    
    return json.loads(array)


In [26]:
chain = (
    prompt
    | ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.8,
        streaming=False,
    )
    | custom_json_parser
)

In [28]:
input_file = "./source/woongjin.jsonl" 
records = []
with open(input_file, "r", encoding="utf-8") as f:
    for line_no, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            rec = json.loads(line)
            if "messages" in rec:
                records.append(rec)
            elif "instruction" in rec and "output" in rec:
                records.append({
                    "messages": [
                        {"role": "user",      "content": rec["instruction"]},
                        {"role": "assistant", "content": rec["output"]}
                    ]
                })
            else:
                print(f"라인 {line_no} 건너뜀: 알 수 없는 포맷")
        except json.JSONDecodeError as e:
            print(f"라인 {line_no} 건너뜀: JSONDecodeError: {e}")

In [29]:
records[:3]

[{'messages': [{'role': 'user',
    'content': '웅진코웨이 정수기를 사용할 때 전원 플러그는 어떤 콘센트에 꽂아야 하나요?'},
   {'role': 'assistant',
    'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]},
 {'messages': [{'role': 'user',
    'content': '손상되었거나 헐거운 전원 콘센트를 사용하면 어떤 위험이 있나요?'},
   {'role': 'assistant',
    'content': '손상되었거나 헐거운 전원 콘센트를 사용하면 감전 및 화재의 위험이 있습니다.'}]},
 {'messages': [{'role': 'user', 'content': '전원선을 끌면서 제품을 이동하면 어떤 위험이 있나요?'},
   {'role': 'assistant',
    'content': '전원선을 끌면서 제품을 이동하면 전원선이 손상되어 감전이나 화재의 위험이 있습니다.'}]}]

In [30]:
def generate_similar_qa(input_text: str, num_questions: int = 5):
    return chain.invoke({
        "context": input_text, "num_questions": num_questions})

In [31]:
all_augmented = []
num_questions = 5

for idx, rec in enumerate(records, start=1):
    try:
        msgs = rec["messages"]
        question = next(m["content"] for m in msgs if m["role"] == "user")
        answer   = next(m["content"] for m in msgs if m["role"] == "assistant")
        context = f"'QUESTION': '{question}', 'ANSWER': '{answer}'"
        aug = generate_similar_qa(context, num_questions)
        print(aug)
        all_augmented.extend(aug)
    except Exception as e:
        print(f"레코드 {idx} 처리 중 오류: {e}")

[{'messages': [{'role': 'user', 'content': '웅진코웨이 정수기를 사용할 때, 어떤 전원 콘센트에 꽂아야 하나요?'}, {'role': 'assistant', 'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}, {'messages': [{'role': 'user', 'content': '정수기를 연결할 때, 전원 플러그는 어떤 규격의 콘센트에 꽂아야 하나요?'}, {'role': 'assistant', 'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}, {'messages': [{'role': 'user', 'content': '웅진코웨이 정수기의 전원 플러그를 꽂을 콘센트는 어떻게 선택해야 하나요?'}, {'role': 'assistant', 'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}, {'messages': [{'role': 'user', 'content': '정수기를 안전하게 사용하기 위해 전원 플러그는 어떤 콘센트에 연결해야 하나요?'}, {'role': 'assistant', 'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}, {'messages': [{'role': 'user', 'content': '웅진코웨이 정수기를 연결할 때 필요한 전원 콘센트의 사양은 무엇인가요?'}, {'role': 'assistant', 'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}]
[{'messages': [{'role': 'user', 'content': '손상이 있거나 느슨한 전원 소켓을 사용하는 것이 위험한 이유는 무엇인가요?'}, {'role': 'assistant', 'content': '손상되었거나 헐거운 전원 콘센트를 사용하

In [32]:
out_file = "./source/woongjin_augmented_dataset.jsonl"
with open(out_file, "w", encoding="utf-8") as f:
    for record in all_augmented:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"{len(all_augmented)}개의 QA 페어를 생성하여 '{out_file}'에 저장했습니다.")

1261개의 QA 페어를 생성하여 './source/woongjin_augmented_dataset.jsonl'에 저장했습니다.


In [33]:
all_augmented[:3]

[{'messages': [{'role': 'user',
    'content': '웅진코웨이 정수기를 사용할 때, 어떤 전원 콘센트에 꽂아야 하나요?'},
   {'role': 'assistant',
    'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]},
 {'messages': [{'role': 'user',
    'content': '정수기를 연결할 때, 전원 플러그는 어떤 규격의 콘센트에 꽂아야 하나요?'},
   {'role': 'assistant',
    'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]},
 {'messages': [{'role': 'user',
    'content': '웅진코웨이 정수기의 전원 플러그를 꽂을 콘센트는 어떻게 선택해야 하나요?'},
   {'role': 'assistant',
    'content': '전원 플러그는 반드시 220V~ 60Hz 전용 콘센트에 꽂아서 사용해야 합니다.'}]}]

In [34]:
len(all_augmented)

1261

In [35]:
import json

input_file  = "./source/woongjin_augmented_dataset.jsonl"  
output_file = "./source/data.jsonl"                        

total_count = 0

with open(input_file,  "r", encoding="utf-8") as f_in, \
    open(output_file, "w", encoding="utf-8") as f_out:

    for line in f_in:
        line = line.strip()
        if not line:
            continue

        try:
            record = json.loads(line)
        except json.JSONDecodeError:
            continue

        messages = record.get("messages", [])
        user_msg = next((m["content"] for m in messages if m.get("role")=="user"), None)
        bot_msg  = next((m["content"] for m in messages if m.get("role")=="assistant"), None)

        if not user_msg or not bot_msg:
            continue

        chat_text = (
            "<|im_start|>user\n"
            f"{user_msg.strip()}"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
            f"{bot_msg.strip()}"
            "<|im_end|>"
        )

        out_entry = {"text": chat_text}
        f_out.write(json.dumps(out_entry, ensure_ascii=False) + "\n")
        total_count += 1

print(f"{total_count}개의 QA 페어를 '{output_file}'로 저장했습니다.")

1261개의 QA 페어를 './source/data.jsonl'로 저장했습니다.
